In [1]:
!pip install torch torchvision opencv-python rembg gfpgan
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
%cd detectron2_repo
!pip install -e .

fatal: destination path 'detectron2_repo' already exists and is not an empty directory.
/content/detectron2_repo
Obtaining file:///content/detectron2_repo
  Preparing metadata (setup.py) ... done
  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.6
    Uninstalling detectron2-0.6:
      Successfully uninstalled detectron2-0.6
  Running setup.py develop for detectron2


In [3]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00


In [4]:
from PIL import Image
from rembg import remove
import cv2
import numpy as np
import torchvision.transforms as T
import torch

def load_image(path):
    return Image.open(path).convert("RGB")

def remove_background(img):
    out = remove(img)
    return Image.fromarray(np.array(out))

def preprocess(img, size=(256, 192)):
    transform = T.Compose([
        T.Resize(size),
        T.ToTensor()
    ])
    return transform(img).unsqueeze(0)  # Shape: [1, 3, H, W]


In [5]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

def setup_segmentor():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
        "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    return DefaultPredictor(cfg)

segmentor = setup_segmentor()

def segment_person(image_np):
    outputs = segmentor(image_np)
    mask = outputs["instances"].pred_masks[0].cpu().numpy().astype(np.uint8)
    return Image.fromarray(mask * 255)


model_final_f10217.pkl: 178MB [00:01, 120MB/s]                           


In [6]:
import torch.nn as nn
import torch.nn.functional as F

class GMM(nn.Module):  # Simplified Thin Plate Spline proxy
    def __init__(self):
        super(GMM, self).__init__()
        self.warp = nn.Sequential(
            nn.Conv2d(6, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 3, 3, padding=1),
            nn.Tanh()
        )

    def forward(self, person, cloth):
        return self.warp(torch.cat([person, cloth], dim=1))


In [7]:
class TryOnSynthesizer(nn.Module):
    def __init__(self):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Conv2d(6, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 3, 3, padding=1),
            nn.Tanh()
        )

    def forward(self, person, warped_cloth):
        return self.gen(torch.cat([person, warped_cloth], dim=1))


In [11]:
def remove_background_rgb(img):
    out = remove(img)
    out_rgb = Image.fromarray(np.array(out)).convert("RGB")
    return out_rgb


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
gmm = GMM().cuda()
tom = TryOnSynthesizer().cuda()
optimizer = torch.optim.Adam(list(gmm.parameters()) + list(tom.parameters()), lr=0.001)

for epoch in range(10):
    person_img = preprocess(load_image('/content/drive/MyDrive/Colab Notebooks/VTO2/person.png')).cuda()
    cloth_img = preprocess(remove_background_rgb(load_image('/content/drive/MyDrive/Colab Notebooks/VTO2/cloth.png'))).cuda()

    warped = gmm(person_img, cloth_img)
    output = tom(person_img, warped)

    loss = F.l1_loss(output, person_img)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch} - Loss: {loss.item():.4f}")


Epoch 0 - Loss: 0.7905
Epoch 1 - Loss: 0.6726
Epoch 2 - Loss: 0.5522
Epoch 3 - Loss: 0.4268
Epoch 4 - Loss: 0.3067
Epoch 5 - Loss: 0.2231
Epoch 6 - Loss: 0.1645
Epoch 7 - Loss: 0.1294
Epoch 8 - Loss: 0.1181
Epoch 9 - Loss: 0.1297


In [15]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [16]:
torch.save({
    'gmm': gmm.state_dict(),
    'tom': tom.state_dict()
}, 'virtual_tryon_model.pth')


In [18]:
# Load the test person and garment images
test_person = preprocess(load_image('/content/drive/MyDrive/Colab Notebooks/VTO2/person.png')).cuda()
test_cloth = preprocess(remove_background_rgb(load_image('/content/drive/MyDrive/Colab Notebooks/VTO2/cloth.png'))).cuda()

# Run inference through your trained model
gmm.eval()
tom.eval()
with torch.no_grad():
    warped_test = gmm(test_person, test_cloth)
    output_test = tom(test_person, warped_test)

# Visualize result
import torchvision.transforms as T
from PIL import Image
to_pil = T.ToPILImage()
result_image = output_test.squeeze(0).cpu().clamp(0, 1)
to_pil(result_image).save('test_tryon_result.png')
to_pil(result_image).show()
